## notebook for exploring similar terms/sentences

Turns tweets/posts into 512d vectors using a pretrained model, after which we use dimensionality reduction algorithms to turn the 512d vectors into 2d. We can then use the 2d vectors to visualise these tweets/posts in an interactive graph together with an analyst (currently using the `bulk` package). It will allow us to highlight snippets that have a particular word in them, and see which other snippets are close by. 

This would help analysts explore similar text snippets, and 

1: Give them a better idea of the size and scope of the topics that they are interested in (denoted by those words)

2: Provide inspiration for other words that could have something to do with that cluster, which can be used to bootstrap the SFLM model, or a spaCy model using `patterns` 

- [x] Load data
- [x] load spacy arabic model
    - Used distiluse-base-multilingual-cased-v1 instead of spacy
- [x] Add spacy model to sklearn pipeline
    - Used huggingface through embetter to get BERT model
- [x] Prep and export dataset to show similar sentences through bulk
    - [x] run text through embedding
    - [x] UMAP to dim reduction
    - [x] run bulk to create a small 2d graph of similar sentences

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime

import pandas as pd
import tentaclio
import embetter
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from embetter.grab import ColumnGrabber
from embetter.text import SentenceEncoder

import umap
import hdbscan
import sklearn.cluster as cluster
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score

from phoenix.common import artifacts, run_params, utils
from phoenix.tag.labelling import prodigy_utils

In [ ]:
# !pip install embetter
# !pip install "embetter[sentence-tfm]"
# !pip install umap-learn hdbscan
# !pip install ipywidgets
# !pip install --upgrade jupyter

In [3]:
utils.setup_notebook_output()
utils.setup_notebook_logging()

LOG:2023-02-03 11:49:45,435 - pid:11 - [/src/phoenix/common/utils.py:24] - root - INFO - Outputting logs within notebook enabled. Set level:INFO.


<RootLogger root (INFO)>

In [4]:
prodigy_dmaps_df_path = f"{artifacts.urls.get_local()}/prodigy/"
tweets_dmaps_path = f"{artifacts.urls.get_local()}/prodigy/dmaps_jordan_tweets.csv"
written_path = "/Users/andrewsutjahjo/git/python/phoenix/local_artifacts//prodigy/dmaps_jordan_tweets-11.csv"
fb_posts_tanzania_path = f"{prodigy_dmaps_df_path}tanzania_facebook_posts_final.parquet"
output_path = f"{artifacts.urls.get_local()}/prodigy/dmaps_jordan_tweets-11.csv"

In [11]:
# wiki_data = f"{artifacts.urls.get_local()}/prodigy/sw_small.txt"
wiki_data = f"{artifacts.urls.get_local()}/prodigy/sw_tiny.txt"
output_path = f"{artifacts.urls.get_local()}/prodigy/sw_small_umap.csv"

In [12]:
with tentaclio.open(wiki_data, "r") as fb:
    df_wiki = pd.read_csv(fb, sep="\n", error_bad_lines=False)

b'Skipping line 507: expected 1 fields, saw 2\nSkipping line 653: expected 1 fields, saw 2\nSkipping line 993: expected 1 fields, saw 2\nSkipping line 1703: expected 1 fields, saw 2\nSkipping line 1736: expected 1 fields, saw 2\nSkipping line 2092: expected 1 fields, saw 2\nSkipping line 2219: expected 1 fields, saw 2\nSkipping line 2360: expected 1 fields, saw 2\nSkipping line 2361: expected 1 fields, saw 2\nSkipping line 3064: expected 1 fields, saw 2\nSkipping line 3441: expected 1 fields, saw 2\nSkipping line 5025: expected 1 fields, saw 2\nSkipping line 5566: expected 1 fields, saw 2\nSkipping line 6050: expected 1 fields, saw 2\nSkipping line 6574: expected 1 fields, saw 2\nSkipping line 6668: expected 1 fields, saw 2\nSkipping line 7725: expected 1 fields, saw 2\nSkipping line 8852: expected 1 fields, saw 2\nSkipping line 8922: expected 1 fields, saw 2\n'


In [13]:
df_wiki.rename({"MKUTANO WA BIASHARA": "text"}, axis=1, inplace=True)

In [14]:
df_wiki

,text
0,▪ Je Ungependa Kupata Mualiko Maalum Kuhudhuri...
1,▪ Nikupe Maelezo Zaidi Ya Namna Ya Ushiriki Wa...
2,Labels: MKUTANO WA BIASHARA
3,"Lakini inakubaliwa hadhi ya ""dini"" katika nchi..."
4,"Kadiri ya hesabu yake, hao mwaka 2005 walikuwa..."
...,...
8167,Mwenyekiti wa Tanzania Saccos For Women Entrep...
8168,"Alisema kuwa TASWE inamatawi 14, katika matawi..."
8169,"Mshereheshaji, Angela Bondo akizungumza katika..."
8170,Rais wa Chama cha wenye viwanda Biashara na Ki...


In [5]:
df = artifacts.dataframes.get(fb_posts_tanzania_path).dataframe

In [6]:
df.groupby("language_from_api").count()

#  df = df[:10]

,phoenix_post_id,platform_id,platform,date,updated,type,text,link,post_url,subscriber_count,total_interactions,video_length_ms,id,image_text,title,caption,description,account_name,account_handle,account_platform_id,account_page_category,account_page_admin_top_country,account_page_description,account_url,account_page_created_date,statistics_actual_like_count,statistics_actual_comment_count,statistics_actual_share_count,statistics_actual_love_count,statistics_actual_wow_count,statistics_actual_haha_count,statistics_actual_sad_count,statistics_actual_angry_count,statistics_actual_care_count,overperforming_score,interaction_rate,underperforming_score,post_created,timestamp_filter,date_filter,year_filter,month_filter,day_filter,medium_type,text_link,text_hash,scrape_url,url_post_id,file_timestamp
language_from_api,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ar,4161,4161,4161,4161,4161,4161,4161,3922,4161,4161,4161,514,4161,313,2430,2434,2430,4161,4156,4161,4161,4161,4161,4161,4159,4161,4161,4161,4161,4161,4161,4161,4161,4161,0,0,0,4161,4161,4161,4161,4161,4161,4161,4161,4161,4161,4161,4161
en,40,40,40,40,40,40,40,40,40,40,40,3,40,16,9,9,10,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,0,0,0,40,40,40,40,40,40,40,40,40,40,40,40
fr,111,111,111,111,111,111,111,111,111,111,111,0,111,2,108,108,106,111,111,111,111,111,111,111,111,111,111,111,111,111,111,111,111,111,0,0,0,111,111,111,111,111,111,111,111,111,111,111,111
und,294,294,294,294,294,294,294,292,294,294,294,1,294,8,276,275,277,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,0,0,0,294,294,294,294,294,294,294,294,294,294,294,294


In [7]:
df[df["language_from_api"] == "und"]

,phoenix_post_id,platform_id,platform,date,updated,type,text,link,post_url,subscriber_count,total_interactions,video_length_ms,language_from_api,id,image_text,title,caption,description,account_name,account_handle,account_platform_id,account_page_category,account_page_admin_top_country,account_page_description,account_url,account_page_created_date,statistics_actual_like_count,statistics_actual_comment_count,statistics_actual_share_count,statistics_actual_love_count,statistics_actual_wow_count,statistics_actual_haha_count,statistics_actual_sad_count,statistics_actual_angry_count,statistics_actual_care_count,overperforming_score,interaction_rate,underperforming_score,post_created,timestamp_filter,date_filter,year_filter,month_filter,day_filter,medium_type,text_link,text_hash,scrape_url,url_post_id,file_timestamp
137,100037423069130-f0c594ec84786b46,1.000374e+29,Facebook,2023-01-27 09:26:38,2023-01-27 12:01:37+00:00,photo,#بلدية_الهري,https://www.facebook.com/photo.php?fbid=896086...,https://www.facebook.com/100037423069130/posts...,99898,1.0,NaN,und,11785489|896086731648754,None,None,None,None,KOURA ONLINE,kouraon,100037423069130,MEDIA_NEWS_COMPANY,LB,\nصفحة إخبارية،اجتماعية،منوعة. مستقلة،لا تنتمي...,https://www.facebook.com/360303224491343,2018-05-13 10:29:18+00:00,1,0,0,0,0,0,0,0,0,NaN,NaN,NaN,2023-01-27 09:26:38+00:00,2023-01-27 09:26:38+00:00,2023-01-27,2023,1,27,photo,#بلدية_الهري-https://www.facebook.com/photo.ph...,f0c594ec84786b46,https://mbasic.facebook.com/100037423069130/po...,896086731648754,2023-01-27 12:29:38.172827+00:00
155,100044236668951-aa5d33c1c81107cb,1.000442e+29,Facebook,2023-01-25 16:42:41,2023-01-27 10:42:51+00:00,photo,"""تجدد"": مستمرّون في النضال إلى جانب أهالي ضحاي...",https://www.facebook.com/kutlattajadod/photos/...,https://www.facebook.com/100044236668951/posts...,101220,85.0,NaN,und,885198|755065189311363,None,None,None,"اجتمعت كتلة ""تجدد"" في مقرها في سن الفيل وأصدرت...",Michel Moawad,michelmoawadofficial,100044236668951,POLITICIAN,LB,"Member of the 🇱🇧 Parliament, proudly represent...",https://www.facebook.com/144651662390924,2013-06-04 06:42:26+00:00,70,7,3,5,0,0,0,0,0,NaN,NaN,NaN,2023-01-25 16:42:41+00:00,2023-01-25 16:42:41+00:00,2023-01-25,2023,1,25,photo,"""تجدد"": مستمرّون في النضال إلى جانب أهالي ضحاي...",aa5d33c1c81107cb,https://mbasic.facebook.com/100044236668951/po...,755065189311363,2023-01-27 12:29:38.172827+00:00
177,100044242578779-5ced73c0afb8d956,1.000442e+29,Facebook,2023-01-27 07:19:56,2023-01-27 11:23:38+00:00,link,⬆️,https://maghapress.blogspot.com/p/blog-page.html,https://www.facebook.com/100044242578779/posts...,128689,3.0,NaN,und,10287848|725574232260656,None,أسعار الدولار وصيرفة والعملات الرقمية,maghapress.blogspot.com,None,آغابرس اخبار صيدا,aghapress1,100044242578779,TOPIC_PUBLISHER,LB,NEWS\nصفحة إخبارية متنوعة,https://www.facebook.com/1415405748740902,2014-06-04 18:10:31+00:00,3,0,0,0,0,0,0,0,0,NaN,NaN,NaN,2023-01-27 07:19:56+00:00,2023-01-27 07:19:56+00:00,2023-01-27,2023,1,27,link,⬆️-https://maghapress.blogspot.com/p/blog-page...,5ced73c0afb8d956,https://mbasic.facebook.com/100044242578779/po...,725574232260656,2023-01-27 12:29:38.172827+00:00
178,100044242578779-66793e9fb0b7e20e,1.000442e+29,Facebook,2023-01-27 06:11:41,2023-01-27 11:23:38+00:00,photo,Antar tours,https://www.facebook.com/photo.php?fbid=725545...,https://www.facebook.com/100044242578779/posts...,128689,1.0,NaN,und,10287848|725545175596895,"‎one or more people, ‎'‎رحله الى الشام rours A...",None,None,None,آغابرس اخبار صيدا,aghapress1,100044242578779,TOPIC_PUBLISHER,LB,NEWS\nصفحة إخبارية متنوعة,https://www.facebook.com/1415405748740902,2014-06-04 18:10:31+00:00,1,0,0,0,0,0,0,0,0,NaN,NaN,NaN,2023-01-27 06:11:41+00:00,2023-01-27 06:11:41+00:00,2023-01-27,2023,1,27,photo,Antar tours-https://www.facebook.com/photo.php...,66793e9fb0b7e20e,https://mbasic.facebook.com/100044242578779/po...,725545175596895,2023-01-27 12:29:38.172827+00:00
198,100044242578779-ee3b8fddbcc03963,1.000442e+29,Facebook,2023-01-27 05:

In [9]:
text_emb_pipeline = make_pipeline(
    ColumnGrabber("text"),
    SentenceEncoder("Davlan/bert-base-multilingual-cased-finetuned-swahili")
)


LOG:2023-02-03 11:50:02,090 - pid:11 - [/usr/local/lib/python3.9/site-packages/sentence_transformers/SentenceTransformer.py:66] - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: Davlan/bert-base-multilingual-cased-finetuned-swahili
LOG:2023-02-03 11:50:05,721 - pid:11 - [/usr/local/lib/python3.9/site-packages/sentence_transformers/SentenceTransformer.py:805] - sentence_transformers.SentenceTransformer - WARNING - No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/Davlan_bert-base-multilingual-cased-finetuned-swahili. Creating a new one with MEAN pooling.


Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/Davlan_bert-base-multilingual-cased-finetuned-swahili were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at /root/.cache/torch/sentence_tr

In [15]:
# embeddings_array = text_emb_pipeline.transform(df)
embeddings_array = text_emb_pipeline.transform(df_wiki)

Batches:   0%|          | 0/256 [00:00<?, ?it/s]

In [ ]:
umap_embeddings = umap.UMAP().fit_transform(embeddings_array)

In [ ]:
umap_embeddings

In [ ]:
umap_embeddings.shape[0]

In [ ]:
df["x"] = umap_embeddings[:,0]
df["y"] = umap_embeddings[:,1]

In [ ]:
with tentaclio.open(output_path, "w") as fb:
    df.to_csv(fb)